#Install/Import - RUN ONCE

In [7]:
!pip install --upgrade nfl_data_py
!pip install -U kaleido
import nfl_data_py as nfl
import plotly.graph_objects as go
import pandas as pd
pd.set_option('display.max_columns', None)

In [8]:
pbp = nfl.import_pbp_data([2024], downcast=True)
desc = nfl.import_team_desc()
ids = nfl.import_ids()
pbp = pbp[pbp['qb_dropback'] == 1.0]

2024 done.
Downcasting floats.


In [9]:
pbp = nfl.import_pbp_data([2024], downcast=True)
ftn = nfl.import_ftn_data([2024], downcast=True)
season_roster = nfl.import_seasonal_rosters([2024])
desc = nfl.import_team_desc()
pbp = pbp[pbp['qb_dropback'] == 1.0]
pbp = pbp[pbp['two_point_attempt'] == 0]
pbp = pd.merge(pbp, season_roster[['player_id', 'player_name']], left_on='passer_id', right_on='player_id')
pbp = pd.merge(pbp, desc[['team_abbr', 'team_color', 'team_color2']], left_on='posteam', right_on='team_abbr')
pbp = pd.merge(pbp, ftn[['nflverse_game_id', 'nflverse_play_id', 'is_drop', 'is_qb_fault_sack', 'is_throw_away', 'is_interception_worthy', 'is_catchable_ball']],
               left_on=['play_id', 'game_id'], right_on=['nflverse_play_id', 'nflverse_game_id'], how='inner')

2024 done.
Downcasting floats.
Downcasting floats.


#Choose QB

In [10]:
pbp = pbp[pbp['player_name'] == 'Justin Herbert'] #Insert QB

#Create Graph - RUN FOR EACH QB BEFORE fig.show()

In [11]:
df=pd.DataFrame()
df['Passer'] = pbp['player_name'].iloc[0]
df.loc[0, 'Dropbacks'] = ((pbp['qb_dropback'] == 1.0).sum())
df.loc[0, 'Sacks'] = (pbp['sack'] == 1.0).sum()
df.loc[0, 'Touchdowns'] = ((pbp['touchdown'] == 1.0) & (pbp['fixed_drive_result'] == 'Touchdown')).sum()
df.loc[0, 'Scrambles'] = (pbp['qb_scramble'] == 1.0).sum()
df.loc[0, 'Interceptions'] = (pbp['interception'] == 1.0).sum()
df.loc[0, 'Complete Pass'] = (pbp['complete_pass'] == 1.0).sum()
df.loc[0, 'Incomplete Pass'] = (pbp['incomplete_pass'] == 1.0).sum()
df.loc[0, 'Negative Gain'] = ((pbp['complete_pass'] == 1.0) & (pbp['yards_gained'] < 0)).sum()
df.loc[0, 'Short Gain'] = ((pbp['complete_pass'] == 1.0) & (pbp['yards_gained'] >= 0) & (pbp['yards_gained'] <= 7)).sum()
df.loc[0, 'Medium Gain'] = ((pbp['complete_pass'] == 1.0) & (pbp['yards_gained'] > 7) & (pbp['yards_gained'] < 20)).sum()
df.loc[0, 'Long Gain'] = ((pbp['complete_pass'] == 1.0) & (pbp['yards_gained'] >= 20)).sum()
df.loc[0, 'Throwaway'] = ((pbp['incomplete_pass'] == 1.0) & (pbp['is_throw_away'] == 1.0)).sum()
df.loc[0, 'Drop'] = ((pbp['incomplete_pass'] == 1.0) & (pbp['is_drop'] == 1.0)).sum()
df.loc[0, 'Uncatchable'] = ((pbp['incomplete_pass'] == 1.0) & (pbp['is_catchable_ball'] == False)).sum()
df.loc[0, 'QB Int'] = (((pbp['interception'] == 1.0) & pbp['is_interception_worthy'] == 1.0)).sum()
df.loc[0, 'Short Scramble'] = ((pbp['qb_scramble'] == 1.0) & (pbp['yards_gained'] >= 0) & (pbp['yards_gained'] <= 5)).sum()
df.loc[0, 'Medium Scramble'] = ((pbp['qb_scramble'] == 1.0) & (pbp['yards_gained'] > 5) & (pbp['yards_gained'] < 10)).sum()
df.loc[0, 'Long Scramble'] = ((pbp['qb_scramble'] == 1.0) & (pbp['yards_gained'] >= 10)).sum()
df.loc[0, 'QB Sack'] = (((pbp['pass_attempt'] == 1.0) & pbp['is_qb_fault_sack'] == 1.0)).sum()
df.loc[0, 'Passing Touchdown'] = ((pbp['complete_pass'] == 1) & (pbp['touchdown'] == 1.0) & (pbp['fixed_drive_result'] == 'Touchdown')).sum()
df.loc[0, 'Rushing Touchdown'] = ((pbp['qb_scramble'] == 1) & (pbp['touchdown'] == 1.0) & (pbp['fixed_drive_result'] == 'Touchdown')).sum()
df.loc[0, 'First Down'] = ((pbp['first_down'] == 1.0)).sum()
df.loc[0, 'First Down Pass'] = ((pbp['first_down'] == 1.0) & (pbp['first_down_pass'] == 1.0)).sum()
df.loc[0, 'First Down Rush'] = ((pbp['first_down'] == 1.0) & (pbp['first_down_rush'] == 1.0)).sum()
df.loc[0, 'First Down Penalty'] = ((pbp['first_down'] == 1.0) & (pbp['first_down_penalty'] == 1.0)).sum()
df.loc[0, 'posteam'] = pbp['posteam'].iloc[0]
df = pd.merge(df, desc[['team_abbr', 'team_color', 'team_color2']], left_on='posteam', right_on='team_abbr')

df['Sack %'] = (df['Sacks'] / df['Dropbacks']) * 100
df['Td %'] = (df['Touchdowns'] / df['Dropbacks']) * 100
df['Scr %'] = (df['Scrambles'] / df['Dropbacks']) * 100
df['Int %'] = (df['Interceptions'] / df['Dropbacks']) * 100
df['Comp %'] = (df['Complete Pass'] / df['Dropbacks']) * 100
df['Inc %'] = (df['Incomplete Pass'] / df['Dropbacks']) * 100
df['NegGain %'] = (df['Negative Gain'] / df['Dropbacks']) * 100
df['Short %'] = (df['Short Gain'] / df['Dropbacks']) * 100
df['Med %'] = (df['Medium Gain'] / df['Dropbacks']) * 100
df['Deep %'] = (df['Long Gain'] / df['Dropbacks']) * 100
df['Thrawy %'] = (df['Throwaway'] / df['Dropbacks']) * 100
df['Drop %'] = (df['Drop'] / df['Dropbacks']) * 100
df['Uncatchable %'] = (df['Uncatchable'] / df['Dropbacks']) * 100
df['QB Int %'] = (df['QB Int'] / df['Dropbacks']) * 100
df['Short Scr %'] = (df['Short Scramble'] / df['Dropbacks']) * 100
df['Med Scr %'] = (df['Medium Scramble'] / df['Dropbacks']) * 100
df['Long Scr %'] = (df['Long Scramble'] / df['Dropbacks']) * 100
df['QB Sack %'] = (df['QB Sack'] / df['Dropbacks']) * 100
df['Pass Td %'] = (df['Passing Touchdown'] / df['Dropbacks']) * 100
df['Rush Td %'] = (df['Rushing Touchdown'] / df['Dropbacks']) * 100
df['FD %'] = (df['First Down'] / df['Dropbacks']) * 100
df['FDPa %'] = (df['First Down Pass'] / df['Dropbacks']) * 100
df['FDR %'] = (df['First Down Rush'] / df['Dropbacks']) * 100
df['FDPe %'] = (df['First Down Penalty'] / df['Dropbacks']) * 100

labels = [
    f"Dropbacks {df['Dropbacks'].iloc[0]:.0f}",
    f"Complete Pass ({df['Comp %'].iloc[0]:.1f}%)",
    f"Incomplete Pass ({df['Inc %'].iloc[0]:.1f}%)",
    f"Interception ({df['Int %'].iloc[0]:.1f}%)",
    f"Scramble ({df['Scr %'].iloc[0]:.1f}%)",
    f"Sack ({df['Sack %'].iloc[0]:.1f}%)",
    f"Touchdown ({df['Td %'].iloc[0]:.1f}%)",
    f"Negative Pass ({df['NegGain %'].iloc[0]:.1f}%)",
    f"[0-7] Yard Pass ({df['Short %'].iloc[0]:.1f}%)",
    f"(7-20) Yard Pass ({df['Med %'].iloc[0]:.1f}%)",
    f"20+ Yard Pass ({df['Deep %'].iloc[0]:.1f}%)",
    f"Throwaway ({df['Thrawy %'].iloc[0]:.1f}%)",
    f"Drop ({df['Drop %'].iloc[0]:.1f}%)",
    f"Uncatchable ({df['Uncatchable %'].iloc[0]:.1f}%)",
    f"QB Fault Int ({df['QB Int %'].iloc[0]:.1f}%)",
    f"[0-5] Yard Scramble ({df['Short Scr %'].iloc[0]:.1f}%)",
    f"(5-10) Scramble ({df['Med Scr %'].iloc[0]:.1f}%)",
    f"10+ Yard Scramble ({df['Long Scr %'].iloc[0]:.1f}%)",
    f"QB Fault Sack ({df['QB Sack %'].iloc[0]:.1f}%)",
    f"Passing Touchdown ({df['Pass Td %'].iloc[0]:.1f}%)",
    f"Rushing Touchdown ({df['Rush Td %'].iloc[0]:.1f}%)",
    f"First Down ({df['FD %'].iloc[0]:.1f}%)",
    f"First Down Pass ({df['FDPa %'].iloc[0]:.1f}%)",
    f"First Down Scramble ({df['FDR %'].iloc[0]:.1f}%)",
    f"First Down Penalty ({df['FDPe %'].iloc[0]:.1f}%)"
]
source = [0,0,0,0,0,0,1,1,1,1,2,2,2,3,4,4,4,5,6,6,0,21,21,21]

target = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]

values = [df['Complete Pass'].iloc[0], df['Incomplete Pass'].iloc[0], df['Interceptions'].iloc[0], df['Scrambles'].iloc[0], df['Sacks'].iloc[0], df['Touchdowns'].iloc[0], df['Negative Gain'].loc[0],
          df['Short Gain'].iloc[0], df['Medium Gain'].iloc[0], df['Long Gain'].iloc[0], df['Throwaway'].iloc[0], df['Drop'].iloc[0], df['Uncatchable'].iloc[0], df['QB Int'].iloc[0],
          df['Short Scramble'].iloc[0], df['Medium Scramble'].iloc[0], df['Long Scramble'].iloc[0], df['QB Sack'].iloc[0], df['Passing Touchdown'].iloc[0], df['Rushing Touchdown'].iloc[0],
          df['First Down'].iloc[0], df['First Down Pass'].iloc[0], df['First Down Rush'].iloc[0], df['First Down Penalty'].iloc[0]]

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=50,
        thickness=20,
        label=labels,
        color = df['team_color'].iloc[0]
),
    link=dict(
        source=source,
        target=target,
        value=values,
        color = df['team_color2'].iloc[0],
        line=dict(color='gainsboro', width=0.5)
    )
)])
fig.update_layout(
    width = 1000,
    height = 600,
    title_text=f"What Happens When {pbp['player_name'].iloc[0]} Drops Back?",
    title_font=dict(size=16, color='black'),
    title_xanchor='center',
    title_x=0.5,
    title_y=0.9,
    title_yanchor='top',
    title_pad=dict(b=10),
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.add_annotation(
    text="X:@MB_NFL, Data: FTN and nfl_data_py",
    xref="paper", yref="paper",
    x=0.05, y=0.05,
    showarrow=False,
    font=dict(size=12, color="black"),
    bgcolor="white",
    bordercolor="black",
    borderwidth=1,
    borderpad=4,
)

#Show Graph

In [6]:
fig.write_image("sankey_chart.png")
fig.show() #Run